In [10]:
import spacerocks

In [12]:
from spacerocks.transforms import calc_E_from_M

ImportError: cannot import name 'calc_E_from_M' from 'spacerocks.transforms' (/Users/kjnapier/spacerocks/spacerocks/transforms/__init__.py)

In [4]:
help(spacerocks.transforms)

AttributeError: module 'spacerocks' has no attribute 'transforms'

In [4]:
%%time
transforms.calc_E_from_M(0.5, 0.5)

NameError: name 'transforms' is not defined

In [5]:
spacerocks.__dict__

NameError: name 'spacerocks' is not defined

In [8]:
help(spacerocks.spacerocks)

Help on module spacerocks.spacerocks in spacerocks:

NAME
    spacerocks.spacerocks

DATA
    __all__ = ['transforms']

FILE
    /Users/kjnapier/anaconda3/envs/kev/lib/python3.11/site-packages/spacerocks/spacerocks.cpython-311-darwin.so


